In [1]:
import pandas as pd
import numpy as np
from hdf5_getters import *
from utils import get_all_files
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [76]:
paths = get_all_files('A')

In [7]:
len(paths)

16566

### Features

In [8]:
def h5_to_csv(flist):
    df = pd.DataFrame(columns=['hotness', 'duration', 'energy', 'tempo', 'year', 
                               'analysis_sample_rate', 'danceability', 'end_of_fade_in', 'key', 
                               'key_confidence', 'loudness', 'mode', 'mode_confidence', 'start_of_fade_out', 
                               'tempo', 'time_sig', 'seg_start', 'seg_pitch', 'seg_timbre', 'seg_loud', 
                           'seg_loud_time', 'seg_loud_start', 'section', 'beats', 'bars', 'tatum', 'weight'])
    index = 0
    for fi in flist:
        h5 = open_h5_file_read(fi)
        term = [i.decode('UTF-8') for i in get_artist_terms(h5)]
        weight = 0
        if 'jazz' in term:
            weight = get_artist_terms_weight(h5)[term.index('jazz')]
        df.loc[index] = [get_song_hotttnesss(h5), get_duration(h5), get_energy(h5), get_tempo(h5), get_year(h5),
                get_analysis_sample_rate(h5), get_danceability(h5), get_end_of_fade_in(h5), get_key(h5), 
                get_key_confidence(h5), get_loudness(h5), get_mode(h5),get_mode_confidence(h5), 
                get_start_of_fade_out(h5), get_tempo(h5), get_time_signature(h5),
                np.average(get_segments_start(h5)), np.average(get_segments_pitches(h5)),
                np.average(get_segments_timbre(h5)), np.average(get_segments_loudness_max(h5)),
                np.average(get_segments_loudness_max_time(h5)), np.average(get_segments_loudness_start(h5)),
                np.average(get_sections_start(h5)), np.average(get_beats_start(h5)), 
                np.average(get_bars_start(h5)), np.average(get_tatums_start(h5)), weight]
        index += 1
        h5.close()
    return df

In [9]:
a = h5_to_csv(paths)

/Users/kelly/tools/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:1110: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/Users/kelly/tools/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### Pandas Load csv and classification

In [14]:
a = pd.read_csv("ge.csv")
a.dropna(axis=0, how='any', inplace=True)
a.to_csv("ge.csv", header=None)

In [ ]:
#a.dropna(axis=0, how='any', inplace=True)             # delete NaN rows
y = np.array([1 if i>0.8 else 0 for i in a.weight])   # set label
X = a.values
sum(a.weight>0.85)

In [66]:
cut = int(len(X) * 0.8)
last = len(X)
trainX = X[0:cut]
trainY = y[0:cut]
testX = X[cut:last]
testY = y[cut:last]

In [68]:
clf2 = RandomForestClassifier(n_estimators=100, max_depth=3,
                              random_state=0)
clf2.fit(trainX, trainY)
scores = cross_val_score(clf2, testX, testY, cv=10)

In [69]:
scores

array([ 0.99393939,  0.93939394,  0.95744681,  0.96960486,  0.9787234 ,
        0.97568389,  0.96048632,  0.95744681,  0.95744681,  1.        ])

In [12]:
def parsePoint(line):
    try:
        values = [float(x) for x in line.split(',')]
    except:
        print(values)
    label = 0
    if values[-1] > 0.85:
        label = 1
    return LabeledPoint(label, values)

In [17]:
a = sc.textFile("ge.csv")

In [18]:
data = a.map(parsePoint)

In [10]:
import pandas as pd
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

In [19]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=5, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.map(lambda vp: (vp[0] - vp[1])**2).reduce(lambda x, y: x + y) /labelsAndPredictions.count()

In [20]:
print("Test error is {}".format(testErr))

Test error is 0.0
